In [ ]:
import pandas as pd

df = pd.read_csv("shirts.csv")
print(df)

   height(cm)  weight(kg) t-shirt size
0         158          58            M
1         163          61            M
2         165          61            L
3         168          66            L


# Intro to Machine Learning
* Supervised machine learning: labeled data (i.e., the dataset has an attribute we are interested in predicting for unseen instances)
    * The attribute we want to predict is called the *class* or the *target*
    * If the class is categorical -> classification task
    * If the class is numeric -> regression task
    * Example algorithm: k nearest neighbors (kNN) classifier and regressor
* Unsupervised machine learning: unlabeled data (i.e., the dataset does not have an attribute we are interested in predicting)
    * E.g., clustering/grouping, associations, outliers/anomalies, trends, etc.
    * Example algorithm: k means clustering

## Supervised ML
* We need to divide a dataset into *training* and *testing* sets
    * We train/build an algorithm/model using a training set
    * We evaluate the algorithm/model using a testing set
    * The training and testing sets are *different*